In [2]:
!pip install xlrd

In [3]:
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
      --------------------------------------- 0.3/12.9 MB ? eta -:--:--
     -------- ------------------------------- 2.9/12.9 MB 11.6 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.9 MB 13.7 MB/s eta 0:00:01
     ------------------------------ -------- 10.2/12.9 MB 15.4 MB/s eta 0:00:01
     ---------------------------------------- 12.9/12.9 MB 14.8 MB/s  0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import xlrd
import nltk
import spacy
import string
import os

nlp = spacy.load("es_core_news_sm")
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
directorio = "../Textos de Prueba/"

In [6]:
def Construye_Grafo():
  nombre_archivo= "NAP.xls"
  libro = xlrd.open_workbook(directorio + nombre_archivo)
  hoja = libro.sheet_by_index(0)
  GrafoFrec = nx.Graph()
  GrafoTiempo = nx.Graph()
  GrafoAsoc = nx.Graph()
  num_filas = hoja.nrows
  estimulo = ""
  lema = ""
  frecuencia = 0
  tiempo = 0
  asociado = 0
  for fila in range(0, num_filas-1):
    celda = hoja.cell(fila,0)
    cadena = celda.value
    if cadena == "--PALABRAS--":
      continue
    elif cadena == "======":
      estimulo = hoja.cell(fila+1,4).value
    elif cadena == "" or cadena == "=":
      continue
    elif hoja.cell(fila,2).value == "":
      continue
    else:
      frecuencia = hoja.cell(fila,1).value
      tiempo = hoja.cell(fila,2).value
      asociado = hoja.cell(fila,3).value
      palabra = hoja.cell(fila,4).value
      estimulo = estimulo.strip()
      lema = str(palabra)
      lema = lema.strip()
      GrafoFrec.add_edge(estimulo, lema, weight=1/float(frecuencia))
      GrafoTiempo.add_edge(estimulo, lema, weight=float(tiempo))
      GrafoAsoc.add_edge(estimulo, lema, weight=100-float(asociado))
  return GrafoFrec, GrafoTiempo, GrafoAsoc



In [7]:
g1, g2, g3 = Construye_Grafo()

In [16]:
nx.write_gexf(g1, "mi_grafo_frecuencia.gexf")
nx.write_gexf(g2, "mi_grafo_tiempo.gexf")
nx.write_gexf(g3, "mi_grafo_asociacion.gexf")

In [8]:
print(g1.edges())

[('abeja', 'miel'), ('abeja', 'picar'), ('abeja', 'picadura'), ('abeja', 'reina'), ('abeja', '='), ('abeja', 'amarillo'), ('abeja', 'flor'), ('abeja', 'polen'), ('abeja', 'panal'), ('abeja', 'zumbido'), ('abeja', 'aguijón'), ('abeja', 'colmena'), ('abeja', 'volar'), ('abeja', 'dolor'), ('abeja', 'insecto'), ('abeja', 'animal'), ('abeja', 'avispa'), ('abeja', 'mosca'), ('abeja', 'raya'), ('abeja', 'alergia'), ('abeja', 'amarillo_negro'), ('abeja', 'bicho'), ('abeja', 'campo'), ('abeja', 'dulce'), ('abeja', 'hormiga'), ('abeja', 'jardín'), ('abeja', 'libar'), ('abeja', 'miope'), ('abeja', 'molestia'), ('abeja', 'natación'), ('abeja', 'pequeño'), ('abeja', 'roncha'), ('abeja', 'trabajador'), ('abeja', 'viaje'), ('abeja', 'viejo'), ('abeja', 'yo'), ('abeja', 'mariposa'), ('abeja', 'pañal'), ('abeja', 'plastilina'), ('abeja', 'reja'), ('miel', 'cacahuate'), ('miel', 'chupón'), ('miel', 'cuchara'), ('miel', 'cuna'), ('miel', 'dulce'), ('miel', 'hot_cakes'), ('miel', 'jabón'), ('miel', 'luna'

In [9]:
def limpia_lematiza(cadena):
  limpiado = ""
  palabras_funcionales = nltk.corpus.stopwords.words('spanish')
  for c in string.punctuation:
    cadena = cadena.replace(c, "")
  cadena = cadena.strip()
  for palabra in cadena.split():
    if palabra not in palabras_funcionales:
      doc = nlp(palabra)
      limpiado += doc[0].lemma_ + " "
  return limpiado

In [10]:
limpia_lematiza("insecto de rayas que produce miel")

'insecto rayas producir miel '

In [11]:
def conceptos(lista,subconjunto):
  datos = []
  if len(lista) <= 100:
    tope = len(lista)
  else:
    tope = 100
  for x in range(0,tope):
    if str(lista[x][0]) not in subconjunto and float(lista[x][1])>0:
      datos.append(str(lista[x][0]))
  return datos

In [12]:
def diccionario_nap(definicion):
  grafo_frec, grafo_tiempo, grafo_asociacion = Construye_Grafo()
  texto = limpia_lematiza(definicion)
  texto = texto.lower()
  tokens = texto.split(" ")
  subconjunto_lemas = []
  for token in tokens:
    if token in grafo_frec.nodes() and token!= "":
      subconjunto_lemas.append(token)
  if len(subconjunto_lemas) > 0:
    resultado_asociacion = nx.betweenness_centrality_subset(grafo_asociacion,subconjunto_lemas,subconjunto_lemas,weight="weight",normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x: x[1], reverse=True)[0:99]
    print("Asociación")

    print(conceptos(encontrado,subconjunto_lemas))
    resultado_asociacion = nx.betweenness_centrality_subset(grafo_frec,subconjunto_lemas,subconjunto_lemas,weight="weight",normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x: x[1], reverse=True)[0:99]
    print("Frecuencia")
    print(conceptos(encontrado,subconjunto_lemas))

    resultado_asociacion = nx.betweenness_centrality_subset(grafo_tiempo,subconjunto_lemas,subconjunto_lemas,weight="weight",normalized=True)
    encontrado = sorted(resultado_asociacion.items(), key=lambda x: x[1], reverse=True)[0:99]
    print("Tiempo")
    print(conceptos(encontrado,subconjunto_lemas))
  else:
    print("Definición muy corta, favor de agregar más datos")
  return


In [13]:
diccionario_nap("animal conocido como el rey de la selva")

Asociación
['león', 'hipopótamo']
Frecuencia
['león', 'mono']
Tiempo
['león', 'víbora']


In [14]:
diccionario_nap("insecto de rayas que produce miel")

Asociación
['abeja']
Frecuencia
['abeja', 'picar', 'araña']
Tiempo
['abeja']


In [15]:
diccionario_nap("Medio de transporte que viaja sobre el agua")

Asociación
['autobús', 'dinero', 'barco', 'naranja']
Frecuencia
['fruto', 'autobús', 'naranja', 'gente', 'mercado', 'botella', 'tierra', 'camión', 'barco', 'vino', 'uva', 'tractor']
Tiempo
['cubeta', 'barco', 'color', 'televisión', 'ruta']
